# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-2. rnn_char_seq

<hr>
``` python 

# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(t_cnt, "loss:", l, "Prediction:", ''.join(result_str))


'''
 1th training... 	 Loss : 2.2902767658233643, 	 Prediction : oouuouu u  u uu 
 2th training... 	 Loss : 2.193434476852417, 	     Prediction : oo oou       o  
 3th training... 	 Loss : 2.090886354446411, 	     Prediction : y   ou          
 4th training... 	 Loss : 1.98934805393219, 	     Prediction : y   ou       ou 
 5th training... 	 Loss : 1.8439476490020752, 	 Prediction : y   ou      you 
 6th training... 	 Loss : 1.7353318929672241, 	 Prediction : y   ou      yo  
 7th training... 	 Loss : 1.6576930284500122, 	 Prediction : yf you      you 
 8th training... 	 Loss : 1.681345820426941, 	     Prediction : yf oou w nt yo  
 9th training... 	 Loss : 1.5772677659988403, 	 Prediction : yf oou wwnt you 
 10th training... 	 Loss : 1.5574511289596558, 	 Prediction : yf you wwnt you

...
...

 2995th training... 	 Loss : 0.9896155595779419, 	 Prediction : if you want you 
 2996th training... 	 Loss : 0.9896155595779419, 	 Prediction : if you want you 
 2997th training... 	 Loss : 0.9896154403686523, 	 Prediction : if you want you 
 2998th training... 	 Loss : 0.9896154403686523, 	 Prediction : if you want you 
 2999th training... 	 Loss : 0.9896154403686523, 	 Prediction : if you want you 
 3000th training... 	 Loss : 0.9896153807640076, 	 Prediction : if you want you 
'''

```

<hr>

In [3]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# sample = " if you want you"
sample   = "만약에 당신이 그 누구와 사랑에 빠지면"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
char2idx

{' ': 9,
 '구': 0,
 '그': 11,
 '누': 15,
 '당': 4,
 '랑': 14,
 '만': 10,
 '면': 13,
 '빠': 12,
 '사': 6,
 '신': 8,
 '약': 2,
 '에': 1,
 '와': 7,
 '이': 5,
 '지': 3}

In [5]:
# hyper parameters
dic_size        = len(char2idx)     # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)     # RNN output size
num_classes     = len(char2idx)     # final output size (RNN or softmax, etc.)
batch_size      = 1                 # one sample data, one batch
sequence_length = len(sample) - 1   # number of lstm rollings (unit #)

dic_size, rnn_hidden_size, num_classes, sequence_length

(16, 16, 16, 20)

In [6]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]    # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]     # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

print('x_data : {}'.format(x_data))
print('y_data : {}'.format(y_data))

x_data : [[10, 2, 1, 9, 4, 8, 5, 9, 11, 9, 15, 0, 7, 9, 6, 14, 1, 9, 12, 3]]
y_data : [[2, 1, 9, 4, 8, 5, 9, 11, 9, 15, 0, 7, 9, 6, 14, 1, 9, 12, 3, 13]]


In [7]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)


In [8]:
time1 = time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        
        cnt = t_cnt+1
        if cnt<20 or cnt%100==0:
            # print(t_cnt, "loss:", l, "Prediction:", ''.join(result_str))
            print(" {cnt}th training... \t Loss : {loss}, \t Prediction : {pred} ".format(
                    cnt  = cnt, 
                    loss = l, 
                    pred = ''.join(result_str)
            ))

time2 = time.time()

 1th training... 	 Loss : 2.7593472003936768, 	 Prediction :                면   면 
 2th training... 	 Loss : 2.58374285697937, 	 Prediction :                      
 3th training... 	 Loss : 2.469827651977539, 	 Prediction :                      
 4th training... 	 Loss : 2.3028340339660645, 	 Prediction : 약      그             
 5th training... 	 Loss : 2.2103445529937744, 	 Prediction : 약             랑    면 
 6th training... 	 Loss : 2.124105215072632, 	 Prediction : 약            랑랑랑     
 7th training... 	 Loss : 2.068150281906128, 	 Prediction : 약에 당   그     랑랑      
 8th training... 	 Loss : 2.0155274868011475, 	 Prediction : 약에 당 이 그      랑      
 9th training... 	 Loss : 1.9337736368179321, 	 Prediction : 약에 당   그             
 10th training... 	 Loss : 1.8768390417099, 	 Prediction : 약에 당신                
 11th training... 	 Loss : 1.8428661823272705, 	 Prediction : 약에 당신이               
 12th training... 	 Loss : 1.793089747428894, 	 Prediction : 약에 당신이 그           면 
 13th trai

In [9]:
chk_processting_time(time1, time2)

처리시간 : 0분 15초 경과되었습니다.


15.541375160217285

In [10]:
predict_sentence = sample[0] + ''.join(result_str)
predict_sentence

'만약에 당신이 그 누구와 사랑에 빠지면'

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>